In [47]:
import pandas as pd
import pyupbit
from Get_allassistant import get_assistant
from GetCandle import get_candle
import matplotlib as plt

indicator_data = get_assistant('KRW-BTC', '30', '200', 2)
candle_datas = []
candle_data = get_candle('KRW-BTC', '30', '200')

def get_macd(candle_datas, loop_cnt):
    macd_list = []
 
    df = pd.DataFrame(candle_datas[0])
    df = df.iloc[::-1]
    df = df['trade_price']

    exp1 = df.ewm(span=12, adjust=False).mean()
    exp2 = df.ewm(span=26, adjust=False).mean()
    macd = exp1 - exp2
    exp3 = macd.ewm(span=9, adjust=False).mean()
 
    for i in range(0, int(loop_cnt)):
        macd_list.append(
            {"type": "MACD", "DT": candle_datas[0][i]['candle_date_time_kst'], "MACD": round(macd[i], 4),
            "SIGNAL": round(exp3[i], 4),
            "OCL": round(macd[i] - exp3[i], 4)})
 
    return macd_list

df = get_macd(candle_datas, 2)
plt.plot(df)
plt.show()



IndexError: list index out of range

In [ ]:
import pyupbit
import numpy as np
import json
import time

def upbit_backTesting(coin, range, day, K):
    k = K
    df = pyupbit.get_ohlcv(coin, count = range, period=1, to=day)
    df['range'] = (df['high'] - df['low']) * k
    df['target'] = df['open'] + df['range'].shift(1)
    fee = 0.0005
    df['ror'] = np.where(df['high'] > df['target'],
                        df['close'] / df['target'] - fee,
                        1)
    df['hpr'] = df['ror'].cumprod()
    df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() * 100
    result = []
    result = df['open']
    return strToJson(df)

def strToJson(df):
    dict1 = {"test":[], "MDD":""}
    for i in range(len(df)):
        temp = {}
        for j in df:
            key = j
            value = str(df[j][i])
            temp[key] = value
        strr = df.index[i]
        temp['datetime'] = str(strr)[5:-3]
        dict1['test'].append(temp)
    dict1['MDD'] = df['dd'].max()
    json_val = json.dumps(dict1, default=str)
    return json_val 

def get_tickers():
    return pyupbit.get_tickers(fiat="KRW")

def get_best_k(coin, range, day):
    best_ror = 0
    best_k = 0
    for k in np.arange(0.1, 1.0, 0.1):
        ror = get_ror(coin, range, day, k)
        if best_ror < ror:
            best_ror = ror
            best_k = k

    print(best_k, best_ror)    
    time.sleep(5)
    for k in np.arange(best_k, (best_k + 0.09), 0.01):
        temp_k = round(k,3)
        ror2 = get_ror(coin, range, day, temp_k)

        if best_ror < ror2:
            best_ror = ror2
            best_k = temp_k
    
    print(best_k, best_ror) 

def get_ror(coin, range, day, k):
    df = pyupbit.get_ohlcv(coin, count = range, period=1, to=day)
    df['range'] = (df['high'] - df['low']) * k
    df['target'] = df['open'] + df['range'].shift(1)
 
    fee = 0.0005
    df['ror'] = np.where(df['high'] > df['target'],
                        df['close'] / df['target'] - fee,
                        1)
 
    ror = df['ror'].cumprod()[-2]
    return ror

json1 = get_best_k("KRW-BTC", 100, '20211108')

upbit_backTesting("KRW-BTC", 10, '20210301')

0.2 1.4366204077971814
0.21 1.448010987598862


TypeError: upbit_backTesting() missing 1 required positional argument: 'K'